# Sistema de recomendação de produtos da Amazon (Automotive_5.json) utilizando SVD para Filtro Colaborativo

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn as sns
import numpy as np
import sklearn as sklearn

pd.set_option('float_format', '{:f}'.format)
reviews = pd.read_json('Automotive_5.json', lines = True)

ValueError: Expected object or value

#### Descrição do Dataset:
- reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
- asin - ID of the product, e.g. 0000013714
- reviewerName - name of the reviewer
- vote - helpful votes of the review
- style - a dictionary of the product metadata, e.g., "Format" is "Hardcover"
- reviewText - text of the review
- overall - rating of the product
- summary - summary of the review
- unixReviewTime - time of the review (unix time)
- reviewTime - time of the review (raw)
- image - images that users post after they have received the product

In [ ]:
display(reviews.head(15))

## Análise e Limpeza dos Dados

### Informações Gerais

In [ ]:
reviews.shape
print("Número de Linhas: ", reviews.shape[0])
print("Número de Colunas: ", reviews.shape[1])

In [ ]:
print('Colunas com valores null: \n',reviews.isnull().sum())

In [ ]:
reviews.describe()

In [ ]:
reviews.info()

In [ ]:
print('Quantidade de Usuários: ', reviews['reviewerID'].nunique())
print('Quantidade de Produtos: ', reviews['asin'].nunique())

#### Atributos que não serão utilizados

In [ ]:
reviews.drop(['reviewerName', 'reviewTime', 'unixReviewTime', 'style', 'verified', 'image' ], axis=1, inplace=True)

#### Distribuição do número de avaliações por produto

In [ ]:
# import plotly.graph_objs as go
# from plotly.offline import init_notebook_mode, plot, iplot
import matplotlib.pyplot as plt
import math

distribution = reviews.groupby('asin')['overall'].count().clip(upper=200)
distribution.hist(figsize=(10,5))
plt.ylabel("Número de Produtos, limitado para 200")
plt.xlabel("Número de Avaliações")
plt.show()

#### Recuperando a média da avaliação e a quantidade de vezes que cada produto aparece

Nós vamos utilizar o "dfMerged" para juntar todos os reviews de cada produto. Dessa forma, uma única tupla irá conter todas as palavras de reviews e descrição para cada produto, fazendo com que o KNN consiga avaliar a similaridade dessas palavras por produto. Note que as colunas que terminam em y representam a contagem de avaliações, reviews e votos. A última coluna representa a avaliação média daquele produto.

In [ ]:
count = reviews.groupby("asin", as_index=False).count()
mean = reviews.groupby("asin", as_index=False).mean()

dfMerged = pd.merge(reviews, count, how='right', on=['asin'])
dfMerged = pd.merge(dfMerged, mean, how='right', on=['asin'])
dfMerged

#### Distribuição das avaliações 

Aqui é interessante notar que a grande maioria dos usuários avalia com mais frequência com nota 5 a sua review. Isso pode ser influenciado pela própria Amazon, que incentiva a avaliação por parte dos usuários e a nota 5 exige menos esforço (não é necessário apontar os defeitos). Seria interessante também avaliar a quantidade de palavras escritas na review por nota, para verificar se a nota 5 possui menos do que as demais.

In [ ]:
data = dfMerged.rename(columns={'reviewerID_y': 'reviews_count', 'summary_x': 'product_summary', 'overall': 'avg_rating'})

reviews['overall'].hist(bins=range(1,7), figsize=(10,5),  align='left', rwidth=0.7)
plt.ylabel("Total of Ratings")
plt.xlabel("Ratings")
plt.show()

## Aplicação dos Métodos de Recomendação

### 1. Collaborative Filtering com SVD (Singular Value Decomposition)

#### Separando apenas o ID do usuário, o ID do produto e a nota
Apenas estes atributos de interesse serão utilizados.

In [ ]:
data_svd = dfMerged[['reviewerID_x', 'asin', 'overall_x', 'overall_y']]
data_svd = data_svd.rename(columns={'reviewerID_x': 'user_id', 'asin': 'product_id', 'overall_x': 'rating', 'overall_y': 'reviews_count'})
data_svd = data_svd[data_svd.reviews_count > 50] # Filtrando apenas para os produtos avaliados mais de 50 vezes
data_svd = data_svd[['user_id', 'product_id', 'rating']]

#### Aplicação do Modelo

In [ ]:
from surprise import Reader, Dataset
from surprise.model_selection import KFold

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(data_svd, reader=reader)

kfold = KFold()

In [ ]:
from surprise import SVD
from surprise.model_selection import cross_validate
import numpy as np

svd = SVD()
resultados = cross_validate(svd, data, measures=['rmse', 'mae'], cv=kfold, return_train_measures=True)
print("RMSE:", resultados['test_rmse'].mean())
print("MAE:", resultados['test_mae'].mean())

#### Tuning de hiperparâmetros

In [ ]:
from surprise.model_selection.search import GridSearchCV

param_grid = { 
    'n_factors': [60, 70, 80, 90, 100],
}

gs_svd = GridSearchCV(SVD, param_grid=param_grid, cv=kfold) # Grid Search SVD
gs_svd.fit(data)
gs_svd.best_params

In [ ]:
tuned_svd = SVD(n_factors=gs_svd.best_params['rmse']['n_factors'])
resultados = cross_validate(tuned_svd, data, measures=['rmse', 'mae'], cv=kfold, return_train_measures=True)
print("RMSE:", resultados['test_rmse'].mean())
print("MAE:", resultados['test_mae'].mean())